In [2]:
import pandas as pd
import json

import csv, sqlite3
from sqlite3 import Error
import time

In [3]:
# df = pd.read_json("propertiesForSale.json")

In [4]:
# len(df["hits"])

In [6]:
def Merge(dict1, dict2):
    return(dict2.update(dict1))

def extractData(df, purpose):
    finalDict = {}
    for j in range(0, len(df["hits"])):
        print(j)
        df1 = df["hits"][j]

        columnList = ["id", "userID", "state", "lat", "long", "purpose", "price", "title", "rooms", "baths", "area", "photoUrl", "contact_name"]
        columnFromJsonList = ["id", "ownerID", "state","_geoloc['lat']", "_geoloc['lng']", "purpose", "price", "title", "rooms", "baths", "area", "coverPhoto['url']", "contactName"]
        
        i =0
        dictList = []
        for i in range(0,len(columnList)):
            
            # dictList.append((columnList[i], df1[columnFromJsonList[i]]))
            if(i == 3):
                #print((columnList[i], df1["_geoloc"]["lat"]))
                dictList.append((columnList[i], df1["_geoloc"]["lat"]))
            elif(i==4):
                # print((columnList[i], df1["_geoloc"]["lng"]))
                dictList.append((columnList[i], df1["_geoloc"]["lng"]))
            elif(i==11):
                # print((columnList[i], df1["coverPhoto"]["url"]))
                dictList.append((columnList[i], df1["coverPhoto"]["url"]))
            else:
                # print((columnList[i], df1[columnFromJsonList[i]]))
                dictList.append((columnList[i], df1[columnFromJsonList[i]]))
         
        with open("{p}.json".format(p=purpose), "a") as outfile:
            if(j==0):
                outfile.write("[")
            outfile.write(json.dumps(dict(dictList)))
       
            if(j!=len(df["hits"])):
                outfile.write(",")
   
                
    
            

In [22]:
def convertToCSV(ip, op):

	with open("./{input}.json".format(input = ip)) as json_file:
		jsondata = json.load(json_file)

	data_file = open("./{output}.csv".format(output=op), 'w', newline='')
	csv_writer = csv.writer(data_file)

	count = 0
	for data in jsondata:
		if count == 0:
			header = data.keys()
			csv_writer.writerow(header)
			count += 1
		csv_writer.writerow(data.values())

	data_file.close()



In [25]:
# "My name is {fname}, I'm {age}".format(fname = "John", age = 36)

extractData(pd.read_json("propertiesForSale.json"), "Sale")




0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


In [26]:
extractData(pd.read_json("propertiesForRent.json"), "Rent")



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


In [27]:
convertToCSV("Sale", "SaleCSV")
convertToCSV("Rent", "RentCSV")

In [18]:


# Create a test DB
def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    finally:
        if conn:
            conn.close()



#create_connection(r"./test.db")

# Putting data
def putInDB(inputFile):
    con = sqlite3.connect("./test.db") # change to 'sqlite:///your_filename.db'
    cur = con.cursor()
    # cur.execute("DROP TABLE for_sale;")
    # cur.execute("CREATE TABLE for_rent (id, userID, state, lat, long, purpose, price, title, rooms, baths, area, photoUrl, contact_name);") # use your column names here
    
    # # cur.execute("ALTER TABLE for_sale DROP COLUMN ")

    with open('./{file}.csv'.format(file=inputFile),'r') as fin: # `with` statement available in 2.5+
        # csv.DictReader uses first line in file for column headings by default
        dr = csv.DictReader(fin) # comma is default delimiter
        to_db = [(i['id'], i['userID'], i['state'], i['lat'],i['long'], i['purpose'],i['price'], i['title'],i['rooms'], i['baths'],i['area'], i['photoUrl'],i['contact_name']) for i in dr]

    cur.executemany("INSERT INTO for_rent (id, userID, state, lat, long, purpose, price, title, rooms, baths, area, photoUrl, contact_name) VALUES (?, ?,?, ?,?, ?,?, ?,?, ?,?, ?,?);", to_db)
    con.commit()
    con.close()
    
# putInDB("SaleCSV")

In [19]:
putInDB("RentCSV")